In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import QasmSimulator
from qiskit.result import marginal_counts
from scipy.spatial.distance import cosine

# 설정
n_qubits = 3
shots = 1024
simulator = QasmSimulator()

# 회로 생성 함수
def make_circuit(theta):
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i, angle in enumerate(theta):
        qc.ry(angle * np.pi, i)
    qc.measure(range(n_qubits), range(n_qubits))
    return qc

# 벡터 변환 함수
def counts_to_vector(counts, n=n_qubits):
    total = sum(counts.values())
    vec = np.zeros(n)
    for bitstring, count in counts.items():
        prob = count / total
        bits = [int(b) for b in bitstring[::-1]]
        vec += np.array(bits) * prob
    return vec

# 유사도 측정 함수
def similarity(v1, v2):
    return 1 - cosine(v1, v2)

# 회로 실행 함수
def run(theta):
    qc = make_circuit(theta)
    tqc = transpile(qc, simulator)
    result = simulator.run(tqc, shots=shots).result()
    return result.get_counts()

# ⬛ 기준 회로
base_theta = np.array([0.25, 0.5, 0.75])

# 조건 C: 기준 회로
counts_C = run(base_theta)
vec_C = counts_to_vector(counts_C)

# 조건 A: 동일 구조 반복
counts_A = run(base_theta)
vec_A = counts_to_vector(counts_A)
sim_A = similarity(vec_C, vec_A)

# 조건 B: 구조 제거 (RY 평균값은 유지하되 순서 무작위화)
shuffled_theta = np.random.permutation(base_theta)
counts_B = run(shuffled_theta)
vec_B = counts_to_vector(counts_B)
sim_B = similarity(vec_C, vec_B)

# 출력
print(f"조건 A (구조 반복) vs C 유사도: {sim_A:.4f}")
print(f"조건 B (무작위 구조) vs C 유사도: {sim_B:.4f}")


In [ ]:
#반복실행 1 :
#조건 A (구조 반복) vs C 유사도: 0.9998
#조건 B (무작위 구조) vs C 유사도: 0.8953
#반복실행2:
#조건 A (구조 반복) vs C 유사도: 0.9999
#조건 B (무작위 구조) vs C 유사도: 0.8731
#반복실행3:
#조건 A (구조 반복) vs C 유사도: 0.9997
#조건 B (무작위 구조) vs C 유사도: 0.4803
#반복실행4:
#조건 A (구조 반복) vs C 유사도: 1.0000
#조건 B (무작위 구조) vs C 유사도: 0.6209
#반복실행 5:
#조건 A (구조 반복) vs C 유사도: 0.9997
#조건 B (무작위 구조) vs C 유사도: 0.9997